In [150]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

## Labelling of train and test data

In [151]:
import os
df_train=pd.read_excel('oneBotTrainingData.xlsx',sheet_name='Train')#oneBotTrainingData.xlsx
df_test=pd.read_excel('GoldenCopyDataFile.xlsx')#GoldenCopyDataFile.xlsx

In [152]:
import re
# Dictionary of english Contractions - move 
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}
# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions
df_train['utterance']=df_train['utterance'].apply(lambda x:expand_contractions(x))

## Cleaning for train data

In [153]:
import re
def clean_text(text):
    text=text.lower()
#     text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

df_train['utterance']=df_train['utterance'].apply(lambda x: clean_text(x))
df_train['utterance']=df_train['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_train['utterance']=df_train['utterance'].str.replace(".", "")

# selecting on > 2 len sentence utterances
df_trainutt=df_train["utterance"].values
df_trainutt
s = pd.Series(df_trainutt)
df_train["utt"]=s.str.split().to_frame()
type(s.str.split().to_frame())
df_train['Length'] = df_train['utt'].str.len()
df_train
#last step
df_train.query('Length > 2', inplace = True) 
df_train=df_train[["utterance","intent"]]
df_train

utterance  \
0                am i required to do mandatory learning    
1      are there any mandatory e learning courses tha...   
2      are there any mandatory elearnings i need to c...   
3      can you provide the list of mandatory training...   
4      can you tell me the mandatory trainings i need...   
...                                                  ...   
18287              how do i delegate timesheet approvals   
18288                delegate timesheet approval in cwms   
18289       can i delegate acpc staff timesheet approval   
18290         delegate contract staff timesheet approval   
18291                     misc fee approval notification   

                                                  intent  
0      training.who_is_required_to_complete_the_manda...  
1      training.who_is_required_to_complete_the_manda...  
2      training.who_is_required_to_complete_the_manda...  
3      training.who_is_required_to_complete_the_manda...  
4      training.who_is_required_to_complete_the_manda...  
...                                                  ...  
18287              cwms.approvals_and_delegation_in_cwms  
18288              cwms.approvals_and_delegation_in_cwms  
18289              cwms.approvals_and_delegation_in_cwms  
18290              cwms.approvals_and_delegation_in_cwms  
18291       cwms.timesheet_and_invoicing_matters_in_cwms  

[17005 rows x 2 columns]

In [154]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','parser'])
nlp.max_length=5000000
# nlp = spacy.load('C:/Users/thiyaneswaranm/Documents/New folder/model_api_gold/en_core_web_sm-2.2.5/en_core_web_sm/en_core_web_sm-2.2.5',disable=['ner','parser'])
def lemmatize_sent(text):
    doc = nlp(text)
    lemm_word=[]
    for token in doc:
        if (token.lemma_.casefold())!=("-pron-"):
    #         print(token, token.lemma_,type(token.lemma_))
            lemm_word.append(token.lemma_)
        else:
            lemm_word.append(token)
    listToStr = ' '.join([str(elem) for elem in lemm_word]) 
    return str(listToStr.lower())
print(lemmatize_sent("my am able min to"))
df_train['utterance']=df_train['utterance'].apply(lemmatize_sent)

my be able min to


## Cleaning for test data

In [155]:
import re
def clean_text(text):
    text=text.lower()
#     text=re.sub('\w*\d\w*','', text)
    #text=re.sub('\n',' ',text)
    #text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

df_test['utterance']=df_test['utterance'].apply(lambda x:expand_contractions(x))

df_test['utterance']=df_test['utterance'].astype(str)
df_test['utterance']=df_test['utterance'].apply(lambda x: clean_text(x))

df_test['utterance']

df_test['utterance']=df_test['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_test['utterance']=df_test['utterance'].str.replace(".", "")

df_test=df_test[["utterance","intent"]]
df_test['utterance']=df_test['utterance'].apply(lemmatize_sent)
df_test

utterance  \
0                                  1bank i d   
1               3 tenet of reward philosophy   
2                                    3 tenet   
3                                    3 tenet   
4                          about be my guest   
...                                      ...   
3495               why need guide min salary   
3496  how often be our minimum salary review   
3497     will bank review our guide min wage   
3498             purpose of set minimum wage   
3499            what affect guide min salary   

                                                 intent  
0     onboarding.when_will_the_new_hire_1bank_id_be_...  
1           compensation.what_is_dbs_rewards_philosophy  
2           compensation.what_is_dbs_rewards_philosophy  
3           compensation.what_is_dbs_rewards_philosophy  
4                         talent_management.be_my_guest  
...                                                 ...  
3495  compensation.how_to_set_and_why_we_need_guided...  
3496  compensation.how_to_set_and_why_we_need_guided...  
3497  compensation.how_to_set_and_why_we_need_guided...  
3498  compensation.how_to_set_and_why_we_need_guided...  
3499  compensation.how_to_set_and_why_we_need_guided...  

[3500 rows x 2 columns]

In [156]:
df_test['utterance'][df_test['utterance']=='']='NONE'

## Labelling

1. more training with more number of epochs
2. more training data 
3. more cleaning steps like spell correction etc.

In [157]:
#df=pd.concat([df_train[['utterance','intent']],df_test[['utterance','intent']]])
le=preprocessing.LabelEncoder()
le.fit(df_train['intent'])

df_train['intent1']=le.transform(df_train['intent'])
mapping=df_train[['intent','intent1']].drop_duplicates().rename(columns={'intent1':'mapping'})

df_test=pd.merge(df_test,mapping,on='intent',how='left').rename(columns={'mapping':'intent1'})

df_test['intent1']=df_test['intent1'].fillna(0).astype(int)

In [158]:
df_train

utterance  \
0                  be i require to do mandatory learning   
1      be there any mandatory e learn course that i n...   
2      be there any mandatory elearning i need to com...   
3      can you provide the list of mandatory training...   
4      can you tell i the mandatory training i need t...   
...                                                  ...   
18287               how do i delegate timesheet approval   
18288                 delegate timesheet approval in cwm   
18289       can i delegate acpc staff timesheet approval   
18290         delegate contract staff timesheet approval   
18291                     misc fee approval notification   

                                                  intent  intent1  
0      training.who_is_required_to_complete_the_manda...      487  
1      training.who_is_required_to_complete_the_manda...      487  
2      training.who_is_required_to_complete_the_manda...      487  
3      training.who_is_required_to_complete_the_manda...      487  
4      training.who_is_required_to_complete_the_manda...      487  
...                                                  ...      ...  
18287              cwms.approvals_and_delegation_in_cwms       18  
18288              cwms.approvals_and_delegation_in_cwms       18  
18289              cwms.approvals_and_delegation_in_cwms       18  
18290              cwms.approvals_and_delegation_in_cwms       18  
18291       cwms.timesheet_and_invoicing_matters_in_cwms       28  

[17005 rows x 3 columns]

In [159]:
mapping

intent  mapping
0      training.who_is_required_to_complete_the_manda...      487
90          training.who_is_my_business_learning_partner      486
111    training.where_do_i_do_complete_my_course_eval...      485
131      training.where_can_i_view_my_completed_learning      484
212    training.where_can_i_locate_my_e-learnings_in_...      483
...                                                  ...      ...
18105      training.how_do_i_check_my_skillsflex_balance      449
18139                        shares.special_shares_award      389
18170         general_benefits.service_recognition_award       50
18209                 compensation.guided_minimum_salary        3
18240  compensation.how_to_set_and_why_we_need_guided...        9

[488 rows x 2 columns]

In [160]:
df_test

utterance  \
0                                  1bank i d   
1               3 tenet of reward philosophy   
2                                    3 tenet   
3                                    3 tenet   
4                          about be my guest   
...                                      ...   
3495               why need guide min salary   
3496  how often be our minimum salary review   
3497     will bank review our guide min wage   
3498             purpose of set minimum wage   
3499            what affect guide min salary   

                                                 intent  intent1  
0     onboarding.when_will_the_new_hire_1bank_id_be_...      211  
1           compensation.what_is_dbs_rewards_philosophy       14  
2           compensation.what_is_dbs_rewards_philosophy       14  
3           compensation.what_is_dbs_rewards_philosophy       14  
4                         talent_management.be_my_guest      423  
...                                                 ...      ...  
3495  compensation.how_to_set_and_why_we_need_guided...        9  
3496  compensation.how_to_set_and_why_we_need_guided...        9  
3497  compensation.how_to_set_and_why_we_need_guided...        9  
3498  compensation.how_to_set_and_why_we_need_guided...        9  
3499  compensation.how_to_set_and_why_we_need_guided...        9  

[3500 rows x 3 columns]

## Save the files

In [162]:
df_train.to_excel('train_onebot_v1.xlsx',index=None)

In [163]:
writer=pd.ExcelWriter('gold_test_v1.xlsx',engine='xlsxwriter')

In [164]:
df_test.to_excel(writer,sheet_name='Test',index=None)
mapping.to_excel(writer,sheet_name='Mapping',index=None)

In [165]:
writer.save()